In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random
import os
import random
import yaml
import cv2
import shutil
import tqdm
import glob
import torch
from ultralytics import YOLO

C:\Users\USER\anaconda3\envs\preparation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = os.getcwd()
img_path = os.path.join(path,"img_data")
images_path = os.path.join(img_path,"All_img_with_human")
annotations_path  = os.path.join(img_path,"All_img_Label_with_human")
test_path = os.path.join(img_path,"test_with_human")

In [3]:
List = ["Top", "Outer", "Pants", "Onepiece", "Skirt", "Else"]
test_data = []
for i in List:
    temp = os.listdir(os.path.join(test_path,i+"_test"))
    for t in temp:
        test_data.append(t.split(".")[0])

all_label_data = [i.split(".")[0] for i in os.listdir(annotations_path)]
label_data = list(set(all_label_data) - set(test_data))



In [4]:
random.seed(1016)
random.shuffle(label_data)
splited_data_num = int(len(label_data)*0.95)
train_num_list = label_data[:splited_data_num]
val_num_list = label_data[splited_data_num:]

In [5]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
def img_preprocessing(img_path):
    # 한글 경로로 되어 있지 않을 때 1번, 한글 경로로 되어 있다면 2번을 사용함.
    # 1번
    #re_img = cv2.imread(img_path)
    # 2번
    img = imread(img_path)
    
    #전처리 부분
    # 여기서 416,416으로 이미지 크기를 조정함.
    # 이밖에 모자이크, 흑백 등의 전처리를 추가하고 싶다면 자유롭게 할 것.
    re_img = cv2.resize(img, (256, 256))
    return re_img
# 파일 위치가 한글 경로가 존재할 때 사용함(없다면 제외하고 해도 상관 없음)
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False

In [6]:
def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [7]:
# image 가져오기
# image 분리하기
# image 저장하기
for i in range(4):
    globals()[f"train_dir{i}"] = os.path.join(img_path,"train"+str(i))
    globals()[f"val_dir{i}"] = os.path.join(img_path,"val"+str(i))
    globals()[f"train_img_dir{i}"] = os.path.join(globals()[f"train_dir{i}"],"images")
    globals()[f"val_img_dir{i}"] = os.path.join(globals()[f"val_dir{i}"],"images")
    globals()[f"train_label_dir{i}"] = os.path.join(globals()[f"train_dir{i}"],"labels")
    globals()[f"val_label_dir{i}"] = os.path.join(globals()[f"val_dir{i}"],"labels")
    make_dir(globals()[f"train_dir{i}"])
    make_dir(globals()[f"val_dir{i}"])
    make_dir(globals()[f"train_img_dir{i}"])
    make_dir(globals()[f"val_img_dir{i}"])
    make_dir(globals()[f"train_label_dir{i}"])
    make_dir(globals()[f"val_label_dir{i}"])

In [8]:
# train_img_dir

In [9]:
for i in train_num_list:
    img_dir = os.path.join(images_path, i+".jpg")
    img = imread(img_dir)
    for j in range(4):
        New_img_dir = os.path.join(globals()[f"train_img_dir{j}"],i+".jpg")
        imwrite(New_img_dir,img)
for i in val_num_list:
    img_dir = os.path.join(images_path, i+".jpg")
    img = imread(img_dir)
    for j in range(4):
        New_img_dir = os.path.join(globals()[f"val_img_dir{j}"],i+".jpg")
        imwrite(New_img_dir,img)

In [10]:
# 나눠서 저장하기
# 라벨 읽고 저장
for file in train_num_list:
    label_path = os.path.join(annotations_path, file+".txt")
    f = open(label_path, 'r')
    data = f.read()
    f.close()
    for j in range(4):
        T_label_path = os.path.join(globals()[f"train_label_dir{j}"], file+".txt")
        h = open(T_label_path,'w')
        h.write(data)
        h.close()
    
for file in val_num_list:
    label_path = os.path.join(annotations_path, file+".txt")
    f = open(label_path, 'r')
    data = f.read()
    f.close()
    for j in range(4):
        T_label_path = os.path.join(globals()[f"val_label_dir{j}"], file+".txt")
        h = open(T_label_path,'w')
        h.write(data)
        h.close()

In [11]:
with open(os.path.join(img_path,"classes.txt"),'r') as f:
    data = f.read()
    classes = data
names = classes.split("\n")[:-1]
nc = len(names)
print(names,nc)

['Top', 'Outer', 'Pants', 'Onepiece', 'Skirt', 'Human'] 6


In [12]:
for i in range(4):
    yaml_data = {'train': globals()[f"train_img_dir{i}"], 'val': globals()[f"val_img_dir{i}"], 'nc': nc, 'names': names}
    yaml_path = os.path.join(path,'data'+str(i)+'.yaml')
    with open(yaml_path,'w') as f:
        yaml.dump(yaml_data,f)
    # 잘 저장되었는 지 확인하기
    with open(yaml_path,'r') as f:
        yaml_file = yaml.load(f, Loader=yaml.FullLoader)
        display(yaml_file)


{'names': ['Top', 'Outer', 'Pants', 'Onepiece', 'Skirt', 'Human'],
 'nc': 6,
 'train': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\train0\\images',
 'val': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\val0\\images'}

{'names': ['Top', 'Outer', 'Pants', 'Onepiece', 'Skirt', 'Human'],
 'nc': 6,
 'train': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\train1\\images',
 'val': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\val1\\images'}

{'names': ['Top', 'Outer', 'Pants', 'Onepiece', 'Skirt', 'Human'],
 'nc': 6,
 'train': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\train2\\images',
 'val': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\val2\\images'}

{'names': ['Top', 'Outer', 'Pants', 'Onepiece', 'Skirt', 'Human'],
 'nc': 6,
 'train': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\train3\\images',
 'val': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\val3\\images'}